In [3]:
import sys
import os
sys.path.insert(1, 'models')

from models import position_roles_model, positions_model, user_roles_model, ous_model

pos = positions_model.get_all_positions(10)


[<models.positions_model.Positions object at 0x000001C042D96330>, <models.positions_model.Positions object at 0x000001C042D95F40>, <models.positions_model.Positions object at 0x000001C042D96180>, <models.positions_model.Positions object at 0x000001C042D96240>, <models.positions_model.Positions object at 0x000001C042D961B0>, <models.positions_model.Positions object at 0x000001C042D960C0>, <models.positions_model.Positions object at 0x000001C042D96120>, <models.positions_model.Positions object at 0x000001C042D96150>, <models.positions_model.Positions object at 0x000001C042D962A0>, <models.positions_model.Positions object at 0x000001C042D95940>]
